# Import Packages

In [2]:
import pandas as pd
import itertools
import warnings
warnings.filterwarnings('ignore')
from tqdm import tqdm_notebook as tqdm

# Load Data

In [3]:
data = pd.read_excel('Stock-Data-Optimization-06-11-2021.xlsm', sheet_name = 'Data')

# Static Method 2

In [4]:
data = data.drop(data.columns[[33, 34, 35, 36, 37, 38, 39, 40, 41, 48]], axis = 1) #drop the calculated column

new_header = data.iloc[0] #grab the first row for the header
data = data[1:] #take the data less the header row
data.columns = new_header #set the header row as the df header

#User Input 8 to 17
lower_bound_sma20 = -5
upper_bound_sma20 = -2
momentum = 0
up_from_day = 0
up_from_open = 9
lower_bound_rsi = 64
upper_bound_rsi = 70
volume_ratio = 1.5
market_cap = 2000
min_event = 30

data['mrkt cap'] = data['mrkt cap'].fillna(0) #Fill Blank space in market cap with 0 

#Creating columns AH to AP 22 to 39
data.loc[(data['20SMA'] > data['50SMA'] ) , 'sma20 > sma50'] = 0
data['sma20 > sma50'] = data['sma20 > sma50'].fillna(1)
data.loc[(data['20SMA %'] < upper_bound_sma20) & (data['20SMA %'] > lower_bound_sma20) , 'sma20 bound check'] = 0
data['sma20 bound check'] = data['sma20 bound check'].fillna(1)
data.loc[(data['Mom'] > momentum) , 'momentum'] = 0
data['momentum'] = data['momentum'].fillna(1)
data.loc[(data['%Up'] > up_from_day) , '8% Daily'] = 0
data['8% Daily'] = data['8% Daily'].fillna(1)
data.loc[(data['% Open'] > up_from_open) , '2% UP'] = 0
data['2% UP'] = data['2% UP'].fillna(1)
data.loc[(data['14minRSI'] < upper_bound_rsi) & (data['14minRSI'] > lower_bound_rsi) , 'RSI (60-72)'] = 0
data['RSI (60-72)'] = data['RSI (60-72)'].fillna(1)
data.loc[(data['Volume Ratio'] < volume_ratio) , 'Volume Ration'] = 0
data['Volume Ration'] = data['Volume Ration'].fillna(1)
data.loc[(data['mrkt cap'] < market_cap) , 'market cap'] = 1
data['market cap'] = data['market cap'].fillna(0)
data.loc[(data['Macd'] == 'Pass') , 'MACD'] = 0
data['MACD'] = data['MACD'].fillna(1)

#Creating all the On, Off scenarios 42,43
switch = list(itertools.product(['ON', 'OFF'], ['ON', 'OFF'], ['ON', 'OFF'], ['ON', 'OFF'], ['ON', 'OFF'], ['ON', 'OFF'],\
                       ['ON', 'OFF'], ['ON', 'OFF'], ['ON', 'OFF']))

df = data[['sma20 > sma50','sma20 bound check', 'momentum', '8% Daily', '2% UP', 'RSI (60-72)', 'Volume Ration', \
           'market cap', 'MACD', 'win', 'upwin', 'Loss', 'update loss' ]] #Selecting relevant columns

#Creating empty lists and Dataframe to be used in iterations 49 to 51
indices = []
col_name = []
result = pd.DataFrame()

#Iteration
for j in tqdm(switch):  
    lis = list(j) #Selecting one scenario at a time for iteration
    new_line = pd.DataFrame({'sma20 > sma50': lis[0], 'sma20percent': lis[1], 'momentum': lis[2], 'percentChange': lis[3],\
          'percentOverOpen': lis[4], 'RSI': lis[5], 'volumeRatio': lis[6], 'mktcap': lis[7], 'MACD': lis[8]}, index=[0])
    #Creating new line with the On Off combination for that iteration
    
    #Determing the index of all the OFFs of the particular instance 61 to 63
    for i in range(len(lis)):
        if lis[i] == "OFF":
            indices.append(i)
    
    #Getting the columns names of OFFs of the particular iteration 66 to 68
    for c in indices:
        col = df.columns[c]
        col_name.append(col)
    
    temp = df.drop(columns = col_name) #Drop all the OFF columns
    df_win = temp[temp['win'] == 1] #Filter the dataset where win is 1
    df_win["count"] = df_win.sum(axis=1) #Create a new column and sum each row
    win_count = df_win["count"].value_counts() #Get the frequency of result at store in the series
    win_filter = win_count.to_frame() #Convert the series to Dataframe
    win_filter = win_filter.reset_index() #Reset index of the dataframe to access both sum number and its frequency
    win_filter = win_filter[win_filter['index'] == 1] #Filter the datframe to get frequency of 1 (win)
    win_filter = win_filter.reset_index(drop = True) #Reset the index so that index of filter value is 0
    
    #If after filter the dataframe is empty it means no win happened in this iteration give me 0 else give me count of win\
    #for that iteration and store the value in variable win 81 to 84
    if win_filter.empty == True:
        win = 0
    else:
        win = win_filter['count'].values[0]
    
    new_line['win'] = win #Add another column and store the win value
    
    #Repeat the process for Update Win, Loss & Update Loss 89 to 132
    df_upwin = temp[temp['upwin'] == 1]
    df_upwin["count"] = df_upwin.sum(axis=1)
    upwin_count = df_upwin["count"].value_counts()
    upwin_filter = upwin_count.to_frame()
    upwin_filter = upwin_filter.reset_index()
    upwin_filter = upwin_filter[upwin_filter['index'] == 1]
    upwin_filter = upwin_filter.reset_index(drop = True)
    
    if upwin_filter.empty == True:
        upwin = 0
    else:
        upwin = upwin_filter['count'].values[0]
    
    new_line['upwin'] = upwin
    
    df_loss = temp[temp['Loss'] == 1]
    df_loss["count"] = df_loss.sum(axis=1)
    loss_count = df_loss["count"].value_counts()
    loss_filter = loss_count.to_frame()
    loss_filter = loss_filter.reset_index()
    loss_filter = loss_filter[loss_filter['index'] == 1]
    loss_filter = loss_filter.reset_index(drop = True)
    
    if loss_filter.empty == True:
        loss = 0
    else:
        loss = loss_filter['count'].values[0]
    
    new_line['Loss'] = loss
    
    df_uploss = temp[temp['update loss'] == 1]
    df_uploss["count"] = df_uploss.sum(axis=1)
    uploss_count = df_uploss["count"].value_counts()
    uploss_filter = uploss_count.to_frame()
    uploss_filter = uploss_filter.reset_index()
    uploss_filter = uploss_filter[uploss_filter['index'] == 1]
    uploss_filter = uploss_filter.reset_index(drop = True)
    
    if uploss_filter.empty == True:
        uploss = 0
    else:
        uploss = uploss_filter['count'].values[0]
    
    new_line['update loss'] = uploss
    
    result = result.append(new_line) #Add the new line in the empty dataframe created in line 49
    indices.clear() #Empty the list for next iteration
    col_name.clear() #Empty the list for next iteration

result['total'] = result['win'] + result['upwin'] + result['Loss'] + result['update loss'] #Get the total number of events
result_filter = result[result['total'] >= min_event] #Filter the iterations which has less than threshold events

#Assign weight to each event 140 to 143
result_filter['winweight'] = result_filter['win'] * 2
result_filter['upwinweight'] = result_filter['upwin'] * 1
result_filter['lossweight'] = result_filter['Loss'] * 2.25
result_filter['uplossweight'] = result_filter['update loss'] * 1.5

#Calculate the Win/Loss Ratio 146 to 149
if (result_filter['lossweight'] + result_filter['uplossweight'] == 0).any():
    result_filter['Win/Loss Ratio'] = 0
else:
    result_filter['Win/Loss Ratio'] = (result_filter['winweight'] + result_filter['upwinweight']) / (result_filter['lossweight'] + result_filter['uplossweight'])
                
result_filter = result_filter.drop(columns = ['total', 'winweight', 'upwinweight', 'lossweight', 'uplossweight'])
#Drop Unneccessary columns

result_sort = result_filter.sort_values(by = ['Win/Loss Ratio'], ascending=False) #Sort the data according Win/Loss Ratio
method2 = result_sort.head() #Pick the top 5 iterations
method2.to_csv('static_method2.csv', index = False) #Save the result in CSV
method2 #Display the result in Ipython

,sma20 > sma50,sma20percent,momentum,percentChange,percentOverOpen,RSI,volumeRatio,mktcap,MACD,win,upwin,Loss,update loss,Win/Loss Ratio
0,OFF,OFF,ON,OFF,ON,ON,OFF,ON,OFF,44,9,16,4,2.309524
0,OFF,OFF,ON,ON,ON,ON,OFF,ON,OFF,44,9,16,4,2.309524
0,ON,OFF,ON,OFF,ON,ON,OFF,ON,OFF,42,7,16,3,2.246914
0,ON,OFF,ON,ON,ON,ON,OFF,ON,OFF,42,7,16,3,2.246914
0,OFF,OFF,OFF,OFF,ON,ON,OFF,ON,OFF,91,19,39,12,1.900709
